In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division
%matplotlib inline 

##### R magic voodoo

In [ ]:
### set up R
def setup_r(): 
    os.environ['R_HOME'] = '/home/lindb/g/R3/lib64/R/' 
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s" % (os.environ['R_HOME'], 
                                                   os.environ['LD_LIBRARY_PATH'])
setup_r()
import readline
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri as pd2ri
pd2ri.activate()
r = robjects.r
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext rpy2.ipython
%matplotlib inline

In [ ]:
%%R
source('/home/lindb/imports.R')
sessionInfo()

In [ ]:
def get_r_series(key):
    s = pd.Series(get_r(key))
    s.index = get_r("names(%s)" % key)
    return s

def get_r_df(key):
    df = pd.DataFrame(get_r(key))
    try:
        rname = get_r("rownames(%s)" % key)
        df.index = rname
    except:
        pass
    
    try:
        cname = get_r("colnames(%s)" % key)
        df.columns = cname
    except:
        pass
    return df

def get_r_list(key):
    

def get_r(key):
    return r(key)

##### dependencies

In [ ]:
pg = pd.read_csv('/home/lindb/teakettle/postmoran/moran_analysis/assignments.txt',index_col=0,sep='\t')
pg.head()

In [ ]:
geno = pd.read_csv('/home/lindb/teakettle/data/moran/moran_infile_final_jan_wconecounts.txt',header=0,sep='\t')
geno = geno[geno.columns[:6]]
geno = geno.loc[np.arange(0,nrow(geno),2),:]
geno.index = geno['samp'].tolist()
geno = geno[geno.columns[1:]]
geno.head()

In [ ]:
# combine data
comb = pd.merge(pg,geno,left_index=True,right_index=True)
comb.head()

In [ ]:
dbh = {}
for samp in comb.index:
    dbh[samp] = comb['dbh_11'][samp]

In [ ]:
comb.to_csv('/home/lindb/teakettle/postmoran/moran_analysis/infile_gts_combined.txt',header=True,sep='\t')

In [ ]:
comb.loc['uc3_new076',['mom','dad']]

In [ ]:
%%R
# read into R
par.mat <- readRDS('/home/lindb/teakettle/data/gibbs/first/end_of_cyc_files/par.mat.RDS')
bframe1 = readRDS('/home/lindb/teakettle/data/moran/dependencies/bframe1.RDS')
bframe2 = readRDS('/home/lindb/teakettle/data/moran/dependencies/bframe2.RDS')
adults <- bframe1[seq(1,nrow(bframe1),2),'samp']
seedlings <- bframe2[seq(1,nrow(bframe2),2),'samp']

# look at fractional parentage using assignment probabilities from moran & clark cycles

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/moran_analysis/seedout'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
%%R
# get all potential parents
DIR = '/home/lindb/teakettle/postmoran/moran_analysis/seedout'
for (k in 1:len(par.mat)){
    # get df with all non-zero probs
    whiches = which(par.mat[[k]] != 0,arr.ind=T)
    df = data.frame(whiches)
    colnames(df) = c('mom','dad') # mom and dad columns
    stopifnot(round(sum(par.mat[[k]][whiches]),2) == 1)
    df[,'prob'] = par.mat[[k]][whiches] # add probs
    # save df
    sname = seedlings[k]
    write.table(df,fname(DIR,sprintf("%s.txt",sname)),sep='\t',row.names=F)
}

In [ ]:
# translate num to samp name
As = get_r_df('adults')
As.loc[599] = 'out-of-plot'
adults = {}
for row in As.index:
    adults[row+1] = As[0][row]
Ss = get_r_df('seedlings')
seeds = {}
for row in Ss.index:
    seeds[row+1] = Ss[0][row]

In [ ]:
# for each seedling get a df with potential parents with probabilities
DIR = '/home/lindb/teakettle/postmoran/moran_analysis/seedout'
pars = {}
for f in fs(DIR):
    samp = op.basename(f).split(".txt")[0]
    df = pd.read_csv(f,sep='\t')
    pars[samp] = df

In [ ]:
pkldump(pars,'/home/lindb/teakettle/postmoran/moran_analysis/pars.pkl')

In [ ]:
pkldump(adults,'/home/lindb/teakettle/postmoran/moran_analysis/adults.pkl')

In [ ]:
pars[samp].head(),sum(pars[samp]['prob'])

In [ ]:
# look at distribution of max probabilities
maxs = []
for samp in pars.keys():
    maxs.append(pars[samp]['prob'].max())
plt.hist(maxs)

In [ ]:
# how does the probabilities shake out over probability classes?
sums = len(maxs)
for thresh in np.arange(0,1,.1):
    l = len([x for x in maxs if x > thresh])
    sums +- l
    print thresh,l
sums

In [ ]:
# replicate moran's method of IDing most likely father/mother pair
mothers = []
fathers = []
for samp in pars:
    row,col,probidx = pars[samp].idxmax()
    row,col,prob = pars[samp].loc[probidx,:]
    mothers.append(row)
    fathers.append(col)

In [ ]:
# check and compare to table(m) from 06a...ipynb
t = table(mothers)
t

In [ ]:
# check and compare using samp names
# print mother samp name, noff, mother dbh
for k,v in t.iteritems():
    if adults[int(k)] == 'out-of-plot':
        print '\t',adults[int(k)],v
    else:
        print adults[int(k)],v, dbh[adults[int(k)]]

In [ ]:
# check and compare to table(f) from 06a...ipynb
t2 = table(fathers)
t2

In [ ]:
# print father samp name, noff, father dbh
for k,v in t2.iteritems():
    if adults[int(k)] == 'out-of-plot':
        print '\t',adults[int(k)],v
    else:
        print adults[int(k)],v, dbh[adults[int(k)]]

In [ ]:
# make sure each df['prob'] sums to 1.0
for samp in pars:
    assert "%.2f" % sum(pars[samp]['prob'].tolist()) == '1.00'

In [ ]:
pars[samp].head()

In [ ]:
# create blank dataframe
momheat = OrderedDict()
dadheat = OrderedDict()
for adult in adults.values():
    momheat[adult] = OrderedDict()
    dadheat[adult] = OrderedDict()
    for juv in seeds.values():
        momheat[adult][juv] = float(0.0)
        dadheat[adult][juv] = float(0.0)
# fill in non-zero probability values
for samp in pars:
    for idx in pars[samp].index:
        mom,dad,prob = pars[samp].loc[idx,]
        mom = adults[int(mom)]
        dad = adults[int(dad)]
        if momheat[mom][samp] == 0.0:
            momheat[mom][samp] = float(prob)
        else:
            momheat[mom][samp] += float(prob)
        if dadheat[dad][samp] == 0.0:
            dadheat[dad][samp] = float(prob)
        else:
            dadheat[dad][samp] += float(prob)

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/moran_analysis/heat'
if not op.exists(DIR):
    mkdir(DIR)
mh = pd.DataFrame(momheat)
mh.to_csv(op.join(DIR,'momheat.txt'),sep='\t')
dh = pd.DataFrame(dadheat)
dh.to_csv(op.join(DIR,'dadheat.txt'),sep='\t')

In [ ]:
mh.head()

In [ ]:
head(dh)

In [ ]:
# assert probs sum to 1
for row in dh.index:
    assert round(sum(dh.loc[row,:].tolist()),2) == 1

In [ ]:
# assert probs sum to 1
for row in mh.index:
    assert round(sum(mh.loc[row,:].tolist()),2) == 1

In [ ]:
%%R
# prep matrices for heatmap
mom = read.csv('~/teakettle/postmoran/moran_analysis/heat/momheat.txt',sep='\t',stringsAsFactors=F)
rownames(mom) <- mom$X
seedlings <- rownames(mom)
adults <- colnames(mom)
mom <- mom[,-1]
dad = read.csv('/home/lindb/teakettle/postmoran/moran_analysis/heat/dadheat.txt',sep='\t',stringsAsFactors=F)
rownames(dad) <- dad$X
dad <- dad[,-1]
m = as.matrix(mom)
d = as.matrix(dad)
# log transform non-zero values
whiches    = which(m != 0)
m[whiches] = log(m[whiches],base=10) #,base=10
whiches    = which(d != 0)
d[whiches] = log(d[whiches],base=10)

In [ ]:
%%R
# determine abline locations
# which seedlings belong to which treatments
bcs = which(grepl('bc',seedlings) == T)
ucs = which(grepl('uc',seedlings) == T)
uns = which(grepl('un',seedlings) == T)
bcseedline = max(bcs) # border line
ucseedline = max(ucs) # border line
# which adults belong to which treatments
abc = which(grepl('bc',adults) == T)
auc = which(grepl('uc',adults) == T)
aun = which(grepl('un',adults) == T)
bcad = max(abc)
ucad = max(auc)
# was going to use this to label margins, but will just do this in Illustrator
bcsat = (diff(range(bcs))/2)+min(bcs)
ucsat = (diff(range(ucs))/2)+min(ucs)
unsat = (diff(range(uns))/2)+min(uns)
abcat = (diff(range(abc))/2)+min(abc)
aucat = (diff(range(auc))/2)+min(auc)
aunat = (diff(range(aun))/2)+min(aun)

In [ ]:
%%R
# plot heatmap for fractional parentage for seedling assignment to a given mother tree
my_palette <- colorRampPalette(rev(c("black","red","orange","yellow","white")))(n = 499)
heatmap(m,col=my_palette,Rowv=NA,Colv=NA,scale="none",
        labRow=FALSE,labCol=FALSE,xlab="Mothers",ylab="Seedlings",margins=c(2,2),
        add.expr = abline(h=c(bcseedline,ucseedline), v=c(bcad,ucad),lwd=.75,col="white")
       )

In [ ]:
%%R
# plot heatmap for fractional parentage for seedling assignment to a given father tree
heatmap(d,col=my_palette,Rowv=NA,Colv=NA,scale="none",
        labRow=FALSE,labCol=FALSE,xlab="Fathers",ylab="Seedlings",margins=c(4,4),
        add.expr = abline(h=c(bcseedline,ucseedline), v=c(bcad,ucad),lwd=.75,col="white")
       )
mtext(c('bc','uc','un'),side=2,at=c(bcsat,ucsat,unsat))

In [ ]:
%%R
# make sure partitions are where I expect them to be
lapply(list(bcs,ucs,uns),FUN=len)

In [ ]:
%%R
# make sure partitions are where I expect them to be
lapply(list(abc,auc,aun),FUN=len)

In [ ]:
# triple check blocking of figure

# ideas to test

* compare all below by treatment and between seeds/pollen (mom to seed, father to seed, father to mother)


1. mean dispersal distance within treatment 
    1. father to seed could be an 'extended/effective dispersal'
1. mean dispersal distance out of treatment - but how to account for proximity of other plots?
    1. can address concern with permutations
1. number of out of plot dispersals 
    1. may be able to account for prox of plots by lumping 'unknown parents' and 'out-of-plot' parents
1. ratio of within to without 
    1. number of dispersals
    1. distances of dispersals


* using vector of potential parents (PP)


1. mean number of PPs within treatment
1. mean number of PPs outside of treatment
1. also by weighting PP by prob

### dependencies

In [ ]:
# transpose dataframes so I can more easily access fractional parentage by key=seedling.ID
dhtrans = dh.transpose()
mhtrans = mh.transpose()

In [ ]:
dhtrans.head()

In [ ]:
comb.head()

In [ ]:
# get utm coordinates for each sample
utm = {}
for samp in comb.index.tolist():
    utm[samp] = [comb['xutm'][samp],
                 comb['yutm'][samp]]

In [ ]:
def collapsedf(df,par):
    # sum probs for a given mom or dad if they appear in more than one pedigree
    # don't actually need this, but oh well
    if nrow(df) == 0:
        return df
    df2 = OrderedDict()
    for x in ['mom','dad','prob']:
        df2[x] = OrderedDict()
    if par == 'mom':
        col = df.columns[0]
        seedcol = df.columns[1]
    else:
        col = df.columns[1]
        seedcol = df.columns[0]
    assert luni(df[seedcol]) == 1
    seed = uni(df[seedcol])[0]
    if luni(df[col]) == nrow(df):
        return df
    else:
        count = 0
        df2[col] = OrderedDict()
        df2[seedcol] = OrderedDict()
        df2['prob'] = OrderedDict()
        for p in uni(df[col]):
            d = df[df[col]==p].copy()
            sums = sum(d['prob'])
            df2[col][count] = p
            df2[seedcol][count] = seed
            df2['prob'][count] = sums
            count += 1
    return pd.DataFrame(df2)
def distance(i1 ,i2):
    # euclidean distance
    xdiff = (utm[i1][0] - utm[i2][0])**2
    ydiff = (utm[i1][1] - utm[i2][1])**2
    return np.sqrt(xdiff+ydiff)
def wtd_dist(seed,par,bytx=False,without=False,byplot=False,bynum=False,weights=False):
    # weight par-off or mother-father distances by probability of parentage
    df = pars[seed].copy()
    if par == 'mom':
        df = df[df['mom'] != 600].copy()
        df.loc[:,'mom'] = [adults[int(x)] for x in df['mom']]
        df.loc[:,'dad'] = [seed for x in range(len(df))]
        df = collapsedf(df,'mom')
    elif par == 'dad':
        df = df[df['dad'] != 600].copy()
        df.loc[:,'dad'] = [adults[int(x)] for x in df['dad']]
        df.loc[:,'mom'] = [seed for x in range(len(df))]
        df = collapsedf(df,'dad')
    else:
        df = df[df['mom'] != 600].copy()
        df = df[df['dad'] != 600].copy()
        df.loc[:,'mom'] = [adults[int(x)] for x in df['mom'].tolist()]
        df.loc[:,'dad'] = [adults[int(x)] for x in df['dad'].tolist()] 
#        return df
    if bytx == True or byplot == True:
        if bytx:
            tx = seed[:2]
            Dict = txsamps
        if byplot:
            tx = seed[:3]
            Dict = plotsamps
#         if luni(df['mom']) == 1: # but what about nrow(df) == 1
        if par == 'mom':
            cols = ['mom']
#         elif luni(df['dad']) == 1:
        elif par == 'dad':
            cols = ['dad']
        else:
            cols = ['mom','dad']
        for col in cols:
            if without == False: # if only looking at samps within the tx/plot
                df = df[df[col].isin(Dict[tx])].copy()
            else:
                df = df[df[col].isin(Dict[tx])==False].copy()
    if bynum == True:
        if weights == False:
            return(nrow(df))
        else:
            probsum = sum(df['prob'])
            return [nrow(df),probsum]
    if nrow(df) == 0:
        return 'NA'
    sums = 0
    probsum = sum(df['prob'])
    df.index = range(nrow(df))
    for idx in df.index:
        ind1,ind2,prob = df.loc[idx,]
        d = distance(ind1,ind2)
        p = prob/probsum
        s = d*p
        sums += s
    return sums

In [ ]:
wtd = {}
for plot in plotseeds:
    wts = []
    ns  = []
    nwtd = []
    wtd[plot] = []
    for seed in plotseeds[plot]:
        n,probsum = wtd_dist(seed,'mom',byplot=True,bynum=True,weights=True)
        ns.append(n)
        wts.append(probsum)
        nwtd.append(n*probsum)
    denom = np.mean(wts)*len(plotadults[plot])
#     num = sum(nwtd)/len(plotseeds[plot])
    num = sum(nwtd)
    print plot, num/denom
    wtd[plot] = num/denom
wtdtx = {}
for plot in plotseeds:
    tx = plot[:2]
    if not tx in wtdtx:
        wtdtx[tx] = []
    wtdtx[tx].append(wtd[plot])
for tx in wtdtx:
    print tx,np.mean(wtdtx[tx])

In [ ]:
wtd = {}
for plot in plotseeds:
    wts = []
    ns  = []
    nwtd = []
    wtd[plot] = []
    for seed in plotseeds[plot]:
        n,probsum = wtd_dist(seed,'par',byplot=True,bynum=True,weights=True)
        ns.append(n)
        wts.append(probsum)
        nwtd.append(n*probsum)
    denom = np.mean(wts)*len(plotadults[plot])
#     num = sum(nwtd)/len(plotseeds[plot])
    num = sum(nwtd)
    print plot, num/denom ,len(plotsamps[plot]),len(plotadults[plot]),len(plotseeds[plot])
    wtd[plot] = num/denom
wtdtx = {}
for plot in plotseeds:
    tx = plot[:2]
    if not tx in wtdtx:
        wtdtx[tx] = []
    wtdtx[tx].append(wtd[plot])
for tx in wtdtx:
    print tx,np.mean(wtdtx[tx])

In [ ]:
wtd = {}
for plot in plotseeds:
    wts = []
    ns  = []
    nwtd = []
    wtd[plot] = []
    for seed in plotseeds[plot]:
        n,probsum = wtd_dist(seed,'par',byplot=True,bynum=True,weights=True)
        ns.append(n)
        wts.append(probsum)
        nwtd.append(n*probsum)
    denom = np.mean(wts)*len(plotadults[plot])
#     num = sum(nwtd)/len(plotseeds[plot])
#     num = sum(nwtd)
    num = sum(nwtd)/len(plotseeds[plot])
#     print plot, num/denom ,len(plotsamps[plot]),len(plotadults[plot]),len(plotseeds[plot])
#     wtd[plot] = num/denom
    wtd[plot] = num
    print plot, num, len(plotadults[plot])
wtdtx = {}
for plot in plotseeds:
    tx = plot[:2]
    if not tx in wtdtx:
        wtdtx[tx] = []
    wtdtx[tx].append(wtd[plot])
for tx in wtdtx:
    print tx,np.mean(wtdtx[tx])

In [ ]:
print seed
df = pars[seed].copy()
df.loc[:,'mom'] = [adults[int(x)] for x in df['mom'].tolist()]
df.loc[:,'dad'] = [adults[int(x)] for x in df['dad'].tolist()]
df[df['mom'].isin(txsamps['un'])==False]

# overal mean dispersal from moran & clark outputs

In [ ]:
# ID most likely father/mother pair for each seedling
assignment = {}
for samp in pars:
    row,col,probidx = pars[samp].idxmax()
    row,col,prob = pars[samp].loc[probidx,:]
    assignment[samp] = [adults[int(row)],adults[int(col)]]
#    print samp,assignment[samp]

#### mother-offspring

In [ ]:
moffdisp = {}
moffdisptx = {}
for samp in assignment:
    plot = samp[:3]
    tx   = samp[:2]
    if not plot in moffdisp:
        moffdisp[plot] = []
    if not tx in moffdisptx:
        moffdisptx[tx] = []
    mom,dad = assignment[samp]
    if mom != 'out-of-plot':
        d = distance(mom,samp)
        moffdisp[plot].append(d)
        moffdisptx[tx].append(d)
for tx in moffdisptx:
    print tx,np.mean(moffdisptx[tx]),np.median(moffdisptx[tx])

In [ ]:
print stats.kruskal(moffdisptx['bc'],moffdisptx['uc'],moffdisptx['un']).pvalue
for i,txi in enumerate(moffdisptx.keys()):
    for j,txj in enumerate(moffdisptx.keys()):
        if i > j:
            print txi,round(np.median(moffdisptx[txi]),2),txj,round(np.median(moffdisptx[txj]),2)
            print stats.kruskal(moffdisptx[txi],moffdisptx[txj]).pvalue

#### mother-father

In [ ]:
mfdisp = {}
mfdisptx = {}
for samp in assignment:
    plot = samp[:3]
    tx = samp[:2]
    if not plot in mfdisp:
        mfdisp[plot] = []
    if not tx in mfdisptx:
        mfdisptx[tx] = []
    mom,dad = assignment[samp]
    if not 'out-of-plot' in [mom,dad]:
        d = distance(mom,dad)
        mfdisp[plot].append(d)
        mfdisptx[tx].append(d)
for tx in mfdisptx:
    print tx,np.mean(mfdisptx[tx]),np.median(mfdisptx[tx])

In [ ]:
print stats.kruskal(mfdisptx['bc'],mfdisptx['uc'],mfdisptx['un']).pvalue
for i,txi in enumerate(mfdisptx.keys()):
    for j,txj in enumerate(mfdisptx.keys()):
        if i > j:
            print txi,txj
            print stats.kruskal(mfdisptx[txi],mfdisptx[txj]).pvalue

In [ ]:
x = stats.kruskal(mfdisptx['bc'],mfdisptx['uc'],mfdisptx['un'])
x.pvalue

#### father-offspring

In [ ]:
doffdisp = {}
doffdisptx = {}
for samp in assignment:
    plot = samp[:3]
    tx = samp[:2]
    if not plot in doffdisp:
        doffdisp[plot] = []
    if not tx in doffdisptx:
        doffdisptx[tx] = []
    mom,dad = assignment[samp]
    if not dad == 'out-of-plot':
        d = distance(samp,dad)
        doffdisp[plot].append(d)
        doffdisptx[tx].append(d)
for tx in doffdisptx:
    print tx,np.mean(doffdisptx[tx]),np.median(doffdisptx[tx])

In [ ]:
print stats.kruskal(doffdisptx['bc'],doffdisptx['uc'],doffdisptx['un']).pvalue
for i,txi in enumerate(doffdisptx.keys()):
    for j,txj in enumerate(doffdisptx.keys()):
        if i > j:
            print txi,txj
            print stats.kruskal(doffdisptx[txi],doffdisptx[txj]).pvalue

# overall mean dispersal from fractional parentage

## $D_{i} = \sum_{j=1}^n (d_{j}*p_{j}) / n$
### $p_{j} = $ 

where $D_i$ = weighted mean distance for $seedling_i$ over each *j* of the *n* non-zero parent-offspring or mother-father probabilities each with distance $d_j$ and probability $p_j$

In [ ]:
# problem is that I can only calc distances for non out-of-plot, 
# therefore it is likely that for any list of possible in-plot parents sum(probs) << 1

In [ ]:
### mother-offspring distances
# can't do for each mother since vector of off probs won't sum to 1
# can only do for each seed
moff = {}
for seed in mh.index.tolist():
    moff[seed] = wtd_dist(seed,'mom')
#     print seed,moff[seed]

In [ ]:
vals = [val for val in moff.values() if not val == 'NA']
print np.mean(vals),np.median(vals)
plt.hist(vals,bins=100)[-1]

In [ ]:
### father-offspring distances
# can't do for each father since vector of off probs won't sum to 1
# can only do for each seed
doff = {}
for seed in mh.index.tolist():
    doff[seed] = wtd_dist(seed,'dad')
#     print seed,doff[seed]

In [ ]:
print np.mean(doff.values()),np.median(doff.values())
plt.hist(doff.values(),bins=15)[-1]

In [ ]:
# mother-father distances
mfs = []
for seed in pars:
    mfs.append( wtd_dist(seed,'par'))

In [ ]:
plt.hist([x for x in mfs if x!='NA'],bins=50)[-1]

# mean dispersal by tx using fractional parentage

In [ ]:
# list of samps by tx
txsamps = {}
txseeds = {}
for pop in uni(comb['pop'].tolist()):
    if not pop == 'us':
        df = comb[comb['pop']==pop].copy()
        txsamps[pop] = df.index.tolist()
        txseeds[pop] = [x for x in df.index.tolist() if 'new' in x]
        print pop,len(txsamps[pop]),len(txseeds[pop])

In [ ]:
# list of samps by plot
plotsamps  = {}
plotseeds  = {}
plotadults = {}
for samp in comb.index.tolist():
    plot = samp[:3]
    if not plot == 'us3':
        if not plot in plotsamps:
            plotsamps[plot] = []
        if not plot in plotseeds:
            plotseeds[plot] = []
        if not plot in plotadults:
            plotadults[plot] = []
        if 'new' in samp:
            plotseeds[plot].append(samp)
        else:
            plotadults[plot].append(samp)
        plotsamps[plot].append(samp) # want adults and seeds
        assert len(plotsamps[plot]) == len(plotseeds[plot]) + len(plotadults[plot])
for plot in plotsamps:
    print plot, len(plotsamps[plot]),len(plotseeds[plot]),len(plotadults[plot])

In [ ]:
'new' in 'bc1_1179','new' in 'bc1_new003'

#### mother-offspring distances by treatment

In [ ]:
txmoff = {}
for tx in txsamps:
    txmoff[tx] = {}
    for samp in txseeds[tx]:
        txmoff[tx][samp] = wtd_dist(samp,'mom')

In [ ]:
vals = {}
for tx in txsamps:
    v = [val for val in txmoff[tx].values() if not val == 'NA']
    vals[tx] = np.mean(v)
    print tx,np.mean(v),np.median(v)
x = np.arange(3)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Mother-offspring distances by treatment')

#### father-offspring distances by treatment

In [ ]:
txdoff = {}
for tx in txsamps:
    txdoff[tx] = {}
    for samp in txseeds[tx]:
        txdoff[tx][samp] = wtd_dist(samp,'dad')

In [ ]:
vals = {}
for tx in txsamps:
    v = [val for val in txdoff[tx].values() if not val == 'NA']
    vals[tx] = np.mean(v)
    print tx,np.mean(v),np.median(v)
x = np.arange(3)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Father-offspring distances by treatment')

#### mother-father distances by treatment

In [ ]:
txmf = {}
for tx in txsamps:
    txmf[tx] = {}
    for samp in txseeds[tx]:
        txmf[tx][samp] = wtd_dist(samp,'par')

In [ ]:
vals = {}
for tx in txsamps:
    v = [val for val in txmf[tx].values() if not val == 'NA']
    vals[tx] = np.mean(v)
    print tx,np.mean(v),np.median(v)
x = np.arange(3)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Mother-father distances by treatment')

# mean dispersal within tx using fractional parentage

### mean dispersal by tx to within tx using fractional parentage
#### mother-offspring distances

In [ ]:
wtxmoff = {}
for tx in txseeds:
    wtxmoff[tx] = {}
    for samp in txseeds[tx]:
        wtxmoff[tx][samp] = wtd_dist(samp,'mom',bytx=True)

In [ ]:
vals = {}
for tx in txsamps:
    v = [val for val in wtxmoff[tx].values() if not val == 'NA']
    vals[tx] = np.mean(v)
    print tx,np.mean(v),np.median(v)
x = np.arange(3)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Mother-offspring distances within treatment')

#### father-offspring distances

In [ ]:
wtxdoff ={}
for tx in txseeds:
    wtxdoff[tx] = {}
    for samp in txseeds[tx]:
        wtxdoff[tx][samp] = wtd_dist(samp,'dad',bytx=True)

In [ ]:
vals = {}
for tx in txsamps:
    v = [val for val in wtxdoff[tx].values() if not val == 'NA']
    vals[tx] = np.mean(v)
    print tx,np.mean(v),np.median(v)
x = np.arange(3)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('father-offspring distances within treatment')

#### mother-father distances

In [ ]:
wtxmf = {}
for tx in txseeds:
    wtxmf[tx] = {}
    for samp in txseeds[tx]:
        wtxmf[tx][samp] = wtd_dist(samp,'par',bytx=True)

In [ ]:
vals = {}
for tx in txsamps:
    v = [val for val in wtxmf[tx].values() if not val == 'NA']
    vals[tx] = np.mean(v)
    print tx,np.mean(v),np.median(v)
x = np.arange(3)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Mother-father distances within treatment')

### mean dispersal by tx to outside of tx using fractional parentage

#### mother-offspring distances

In [ ]:
otxmoff = {}
for tx in txseeds:
    otxmoff[tx] = {}
    for samp in txseeds[tx]:
        otxmoff[tx][samp] = wtd_dist(samp,'mom',bytx=True,without=True)

In [ ]:
vals = {}
for tx in txsamps:
    v = [val for val in otxmoff[tx].values() if not val == 'NA']
    vals[tx] = np.mean(v)
    print tx,np.mean(v),np.median(v)
x = np.arange(3)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Mother-offspring distances outside of treatment')

#### father-offspring distances

In [ ]:
otxdoff = {}
for tx in txseeds:
    otxdoff[tx] = {}
    for samp in txseeds[tx]:
        otxdoff[tx][samp] = wtd_dist(samp,'dad',bytx=True,without=True)

In [ ]:
vals = {}
for tx in txsamps:
    v = [val for val in otxdoff[tx].values() if not val == 'NA']
    vals[tx] = np.mean(v)
    print tx,np.mean(v),np.median(v)
x = np.arange(3)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Father-offspring distances outside of treatment')

#### mother-father distances

In [ ]:
otxmf = {}
for tx in txseeds:
    otxmf[tx] = {}
    for samp in txseeds[tx]:
        otxmf[tx][samp] = wtd_dist(samp,'par',bytx=True,without=True)

In [ ]:
vals = {}
for tx in txsamps:
    v = [val for val in otxmf[tx].values() if not val == 'NA']
    vals[tx] = np.mean(v)
    print tx,np.mean(v),np.median(v)
x = np.arange(3)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Mother-father distances outside of treatment')

# mean dispersal within plot using fractional parentage

#### mother-offspring distances

In [ ]:
wplotmoff = {}
for plot in plotsamps:
    wplotmoff[plot] = {}
    for samp in plotseeds[plot]:
        wplotmoff[plot][samp] = wtd_dist(samp,'mom',byplot=True)

In [ ]:
vals = {}
for plot in plotsamps:
    v = [val for val in wplotmoff[plot].values() if not val == 'NA']
    vals[plot] = np.mean(v)
    print plot,np.mean(v),np.median(v)

plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Mother-offspring distances within plot')

#### father-offspring distances

In [ ]:
wplotdoff = {}
for plot in plotsamps:
    wplotdoff[plot] = {}
    for samp in plotseeds[plot]:
        wplotdoff[plot][samp] = wtd_dist(samp,'dad',byplot=True)

In [ ]:
vals = {}
for plot in plotsamps:
    v = [val for val in wplotdoff[plot].values() if not val == 'NA']
    vals[plot] = np.mean(v)
    print plot,np.mean(v),np.median(v)

plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Father-offspring distances within plot')

#### mother-father distances

In [ ]:
wplotmf = {}
for plot in plotsamps:
    wplotmf[plot] = {}
    for samp in plotseeds[plot]:
        wplotmf[plot][samp] = wtd_dist(samp,'par',byplot=True)

In [ ]:
vals = {}
for plot in plotsamps:
    v = [val for val in wplotmf[plot].values() if not val == 'NA']
    vals[plot] = np.mean(v)
    print plot,np.mean(v),np.median(v)

plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Mother-Father distances within plot')

# mean dispersal by plot to out-of-plot

#### mother-offspring distances

In [ ]:
oplotmoff = {}
for plot in plotsamps:
    oplotmoff[plot] = {}
    for samp in plotseeds[plot]:
        oplotmoff[plot][samp] = wtd_dist(samp,'mom',byplot=True,without=True)

In [ ]:
vals = {}
for plot in plotsamps:
    v = [val for val in oplotmoff[plot].values() if not val == 'NA']
    vals[plot] = np.mean(v)
    print plot,np.mean(v),np.median(v)

plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Mother-offspring distances outside of plot')

#### father-offspring distances

In [ ]:
oplotdoff = {}
for plot in plotsamps:
    oplotdoff[plot] = {}
    for samp in plotseeds[plot]:
        oplotdoff[plot][samp] = wtd_dist(samp,'dad',byplot=True,without=True)

In [ ]:
vals = {}
for plot in plotsamps:
    v = [val for val in oplotdoff[plot].values() if not val == 'NA']
    vals[plot] = np.mean(v)
    print plot,np.mean(v),np.median(v)

plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)), list(vals.keys()))
plt.title('Father-offspring distances outside of plot')

# number of dispersals within plot

#### mother-offspring numbers

In [ ]:
nmoff = {}
for plot in plotseeds:
    nmoff[plot] = []
    for samp in plotseeds[plot]:
        nmoff[plot].append(wtd_dist(samp,'mom',byplot=True,bynum=True))

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in nmoff[plot]:
        lst.append(nmoff[plot][samp])
    vals[plot] = np.mean(lst)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-offspring numbers within plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in nmoff[plot]:
        lst.append(nmoff[plot][samp])
    vals[plot] = np.mean(lst)/len(plotseeds[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-offspring numbers within plot scaled by nseeds/plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in nmoff[plot]:
        lst.append(nmoff[plot][samp])
    vals[plot] = np.mean(lst)/len(plotadults[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-offspring numbers within plot scaled by nadults/plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in nmoff[plot]:
        lst.append(nmoff[plot][samp])
    vals[plot] = np.mean(lst)/(len(plotadults[plot])*len(plotseeds[plot]))
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-offspring numbers within plot scaled by nadults/plot')

#### father-offspring numbers

In [ ]:
ndoff = {}
for plot in plotseeds:
    ndoff[plot] = []
    for samp in plotseeds[plot]:
        ndoff[plot].append(wtd_dist(samp,'dad',byplot=True,bynum=True))

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in ndoff[plot]:
        lst.append(ndoff[plot][samp])
    vals[plot] = np.mean(lst)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Father-offspring numbers within plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in ndoff[plot]:
        lst.append(ndoff[plot][samp])
    vals[plot] = np.mean(lst)/len(plotseeds[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Father-offspring numbers within plot scaled by nseeds/plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in ndoff[plot]:
        lst.append(ndoff[plot][samp])
    vals[plot] = np.mean(lst)/len(plotadults[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Father-offspring numbers within plot scaled by nadults/plot')

#### mother-father numbers

In [ ]:
wnmf = {}
for plot in plotseeds:
    wnmf[plot]=[]
    for samp in plotseeds[plot]:
        wnmf[plot].append(wtd_dist(samp,'par',byplot=True,bynum=True))

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in wnmf[plot]:
        lst.append(wnmf[plot][samp])
    vals[plot] = np.mean(lst)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-father numbers within plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in wnmf[plot]:
        lst.append(wnmf[plot][samp])
    vals[plot] = np.mean(lst)/len(plotseeds[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-father numbers within plot scaled by nseeds/plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in wnmf[plot]:
        lst.append(wnmf[plot][samp])
    vals[plot] = np.mean(lst)/len(plotadults[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-father numbers within plot scaled by nadults/plot')

# number of dispersals outside of plot

#### mother-offspring numbers

In [ ]:
onmoff = {}
for plot in plotseeds:
    onmoff[plot]=[]
    for samp in plotseeds[plot]:
        onmoff[plot].append(wtd_dist(samp,'mom',byplot=True,without=True,bynum=True))

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in onmoff[plot]:
        lst.append(onmoff[plot][samp])
    vals[plot] = np.mean(lst)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-offspring numbers outside of plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in onmoff[plot]:
        lst.append(onmoff[plot][samp])
    vals[plot] = np.mean(lst)/len(plotseeds[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-offspring numbers outside of plot scaled by nseeds in plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in onmoff[plot]:
        lst.append(onmoff[plot][samp])
    vals[plot] = np.mean(lst)/len(plotadults[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-offspring numbers outside of plot scaled by nadults in plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in onmoff[plot]:
        lst.append(onmoff[plot][samp])
    sums = sum([len(plotadults[p]) for p in plotadults if p != plot])
    vals[plot] = np.mean(lst)/sums
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-offspring numbers outside of plot scaled by nadults outside of plot')

#### father-offspring numbers

In [ ]:
ondoff = {}
for plot in plotseeds:
    ondoff[plot]=[]
    for samp in plotseeds[plot]:
        ondoff[plot].append(wtd_dist(samp,'mom',byplot=True,without=True,bynum=True))

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in ondoff[plot]:
        lst.append(ondoff[plot][samp])
    vals[plot] = np.mean(lst)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Father-offspring numbers outside of plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in ondoff[plot]:
        lst.append(ondoff[plot][samp])
    vals[plot] = np.mean(lst)/len(plotseeds[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Father-offspring numbers outside of plot scaled by nseeds in plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in ondoff[plot]:
        lst.append(ondoff[plot][samp])
    vals[plot] = np.mean(lst)/len(plotadults[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Father-offspring numbers outside of plot scaled by nadults in plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in ondoff[plot]:
        lst.append(ondoff[plot][samp])
    sums = sum([len(plotadults[p]) for p in plotadults if p != plot])
    vals[plot] = np.mean(lst)/sums
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Father-offspring numbers outside of plot scaled by nadults outside of plot')

#### mother-father numbers

In [ ]:
onmf = {}
for plot in plotsamps:
    onmf[plot] = []
    for samp in plotseeds[plot]:
        onmf[plot].append(wtd_dist(samp,'par',byplot=True,without=True,bynum=True))

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in onmf[plot]:
        lst.append(onmf[plot][samp])
    vals[plot] = np.mean(lst)
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-Father numbers outside of plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in onmf[plot]:
        lst.append(onmf[plot][samp])
    vals[plot] = np.mean(lst)/len(plotseeds[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-Father numbers outside of plot scaled by nseeds in plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in onmf[plot]:
        lst.append(onmf[plot][samp])
    vals[plot] = np.mean(lst)/len(plotadults[plot])
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-Father numbers outside of plot scaled by nadults in plot')

In [ ]:
vals = {}
for plot in plotsamps:
    lst = []
    for samp in onmf[plot]:
        lst.append(onmf[plot][samp])
    sums = sum([len(plotadults[p]) for p in plotadults if p != plot])
    vals[plot] = np.mean(lst)/sums
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Mother-Father numbers outside of plot scaled by nadults outside of plot')

# ratio of number of dispersals within to without

#### mother-offspring

In [ ]:
rmoff = {}
rmofftx = {}
for plot in plotsamps:
    tx = plot[:2]
    if not tx in rmofftx:
        rmofftx[tx] = []
    if sum(onmoff[plot]) == 0 or sum(nmoff[plot]) == 0:
        rmoff[plot] = 0
    else:
        rmoff[plot] = np.mean(nmoff[plot]) / np.mean(onmoff[plot])
#    print plot, rmoff[plot]
    rmofftx[tx].append(rmoff[plot])
for tx in rmofftx:
    print tx,np.mean(rmofftx[tx])

In [ ]:
vals = rmoff
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Ratio of Mother-offspring numbers inside:outside')

#### father-offspring

In [ ]:
rdoff = {}
rdofftx = {}
for plot in plotsamps:
    tx = plot[:2]
    if not tx in rdofftx:
        rdofftx[tx] = []
    if sum(ondoff[plot]) == 0 or sum(ndoff[plot]) == 0:
        rdoff[plot] = 0
    else:
        rdoff[plot] = np.mean(ndoff[plot]) / np.mean(ondoff[plot])
#    print plot, rdoff[plot]
    rdofftx[tx].append(rdoff[plot])
for tx in rdofftx:
    print tx,np.mean(rdofftx[tx])

In [ ]:
vals = rdoff
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Ratio of Father-offspring numbers inside:outside')

#### mother-father

In [ ]:
rmf = {}
rmftx = {}
for plot in plotsamps:
    tx = plot[:2]
    if not tx in rmftx:
        rmftx[tx] = []
    if sum(onmf[plot]) == 0 or sum(wnmf[plot]) == 0:
        rmf[plot] = 0
    else:
        rmf[plot] = np.mean(wnmf[plot]) / np.mean(onmf[plot])
#     print plot, rmf[plot]
    rmftx[tx].append(rmf[plot])
for tx in rmftx:
    print tx,np.mean(rmftx[tx])

In [ ]:
vals = rmf
plt.bar(range(len(vals)),list(vals.values()))[-1]
plt.xticks(range(len(vals)),list(vals.keys()))
plt.title('Ratio of Father-mother numbers inside:outside')

# what I think I can do
1. father-mother distance greater/smaller than expected by chance
    1. null would be distribution of all in-plot adults to any adults
1. seedling-mother distance greater/smaller than expected by chance
    1. null would be distribution of all seedling-mother distances
1. father-seedling distances "" ""
    1. "" ""

# pollen pool heterogeneity

In [ ]:
# get a list of seedlings per mom with potential fathers
moms = {}
for seed in assignment:
    mom,dad = assignment[seed]
    if not mom == 'out-of-plot':
        if not dad == 'out-of-plot':
            if not mom in moms:
                moms[mom] = {}
                moms[mom]['seeds'] = []
                moms[mom]['dads'] = []
            moms[mom]['seeds'].append(seed)
            moms[mom]['dads'].append(dad)

In [ ]:
# include only those with greater than one seed
nums = []
for mom in moms:
    l = len(moms[mom]['seeds'])
    if l > 1:
        print mom, l
        nums.append(l)

In [ ]:
plt.hist(nums)[-1]